## Step 1: Imports

In [1]:
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, ModernBertForSequenceClassification, DataCollatorWithPadding
from sklearn.model_selection import train_test_split
from datasets import Dataset

2025-08-11 01:40:25.105184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754876425.463155      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754876425.568698      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Step 2: Load train and test data

In [3]:
train               = pd.read_csv('/kaggle/input/map-charting-student-math-misunderstandings/train.csv')
test                = pd.read_csv('/kaggle/input/map-charting-student-math-misunderstandings/test.csv')

## Step 3: Preapring label encoders for both models

In [4]:
le                  = LabelEncoder()
train.Misconception     = train.Misconception.fillna('NA')
train['target']   = train.Category + ':' +train.Misconception
train['label']    = le.fit_transform(train['target'])

n_classes = len(le.classes_)
print(f"Train shape: {train.shape} with {n_classes} target classes")

Train shape: (36696, 9) with 65 target classes


In [5]:
idx = train.apply(lambda row: row.Category.split('_')[0],axis=1)=='True'
correct = train.loc[idx].copy()
correct['c'] = correct.groupby(['QuestionId','MC_Answer']).MC_Answer.transform('count')
correct = correct.sort_values('c',ascending=False)
correct = correct.drop_duplicates(['QuestionId'])
correct = correct[['QuestionId','MC_Answer']]
correct['is_correct'] = 1

train = train.merge(correct, on=['QuestionId','MC_Answer'], how='left')
train.is_correct = train.is_correct.fillna(0)

In [6]:
test = test.merge(correct, on=['QuestionId','MC_Answer'], how='left')
test.is_correct = test.is_correct.fillna(0)

In [7]:
def format_input(row):
    x = "This answer is correct."
    if not row['is_correct']:
        x = "This is answer is incorrect."
    return (
        f"Question: {row['QuestionText']}\n"
        f"Answer: {row['MC_Answer']}\n"
        f"{x}\n"
        f"Student Explanation: {row['StudentExplanation']}"
    )

test['text'] = test.apply(format_input,axis=1)

In [8]:
ds_test = Dataset.from_pandas(test)

In [9]:
model = AutoModelForSequenceClassification.from_pretrained("/kaggle/input/qwen3-8b-map-competition/MAP_EXP_16_FULL", device_map="cuda:0", torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
model

Qwen3ForSequenceClassification(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 4096)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((4096,), eps=1e-06)
        (post_attentio

In [11]:
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/qwen3-8b-map-competition/MAP_EXP_16_FULL")
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

ds_test = ds_test.map(tokenize, batched=True)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [12]:
test_args = TrainingArguments(
    output_dir="./",
    do_train=False,
    do_predict=True,
    per_device_eval_batch_size=1, # Adjust as needed
    bf16=False, # TRAIN WITH BF16 IF LOCAL GPU IS NEWER GPU          
    fp16=True,
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=test_args,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer)
)

predictions = trainer.predict(ds_test)

predictions.predictions

/tmp/ipykernel_19/1882862596.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


array([[-0.5917969 , -1.203125  , -1.8378906 , -5.1210938 , -1.4160156 ,
        -1.9462891 , -3.0878906 , -1.1455078 , -1.5546875 , -2.0859375 ,
        -2.0722656 , -1.9472656 , -2.4023438 ,  0.09143066, -2.3847656 ,
        -2.7363281 , -2.        , -1.7636719 , -2.5761719 , -3.2539062 ,
        -1.1259766 , -2.0507812 , -2.1679688 , -1.7246094 , -1.4677734 ,
        -3.2558594 ,  0.16394043, -2.5898438 , -3.9628906 , -2.0585938 ,
        -1.9960938 , -1.1503906 , -2.0117188 , -3.2011719 , -1.9042969 ,
        -1.4375    , -1.8359375 , 11.9921875 , -1.2832031 , -0.5957031 ,
        -4.6523438 , -1.6435547 ,  2.0371094 , -0.62597656,  0.7109375 ,
         0.43188477, -0.81933594,  4.2773438 , -2.53125   ,  0.9970703 ,
         1.4169922 , -2.5703125 , -0.29833984, -1.1210938 , -1.7441406 ,
        -0.5024414 ,  0.6665039 , -1.3164062 , -0.3154297 ,  0.8051758 ,
         0.84814453, -2.3144531 ,  0.9824219 ,  0.6645508 ,  6.9726562 ],
       [ 1.1054688 , -2.2421875 , -1.6552734 , -1.

In [13]:
top3           = np.argsort(-predictions.predictions, axis=1)[:, :3]
flat_top3      = top3.flatten()
decoded_labels = le.inverse_transform(flat_top3)
top3_labels_cat    = decoded_labels.reshape(top3.shape)
top3_labels_cat

array([['True_Correct:NA', 'True_Neither:NA',
        'True_Misconception:Incomplete'],
       ['False_Misconception:WNB', 'False_Misconception:Incomplete',
        'False_Neither:NA'],
       ['True_Neither:NA', 'True_Correct:NA',
        'True_Misconception:Shorter_is_bigger']], dtype=object)

In [14]:
joined_preds = []

for preds in top3_labels_cat:
    joined_preds.append(" ".join(preds))



# Save submission
sub = pd.DataFrame({
    "row_id": test.row_id.values,
    "Category:Misconception": joined_preds
})
sub.to_csv("submission.csv", index=False)
sub.head()

,row_id,Category:Misconception
0,36696,True_Correct:NA True_Neither:NA True_Misconcep...
1,36697,False_Misconception:WNB False_Misconception:In...
2,36698,True_Neither:NA True_Correct:NA True_Misconcep...
